In [112]:
import pandas as pd
import numpy as np
import math
import sys
from scipy.sparse.linalg import svds

In [85]:
cols = ['userId', 'movieId', 'rating']

ratings = pd.read_csv(r'C:\Users\Korisnik\Desktop\Work\DataSets\ratings_small.csv', usecols = cols)
print(ratings)

        userId  movieId  rating
0            1       31     2.5
1            1     1029     3.0
2            1     1061     3.0
3            1     1129     2.0
4            1     1172     4.0
...        ...      ...     ...
99999      671     6268     2.5
100000     671     6269     4.0
100001     671     6365     4.0
100002     671     6385     2.5
100003     671     6565     3.5

[100004 rows x 3 columns]


In [86]:
ratings_set = ratings.values
ratings_set

array([[1.000e+00, 3.100e+01, 2.500e+00],
       [1.000e+00, 1.029e+03, 3.000e+00],
       [1.000e+00, 1.061e+03, 3.000e+00],
       ...,
       [6.710e+02, 6.365e+03, 4.000e+00],
       [6.710e+02, 6.385e+03, 2.500e+00],
       [6.710e+02, 6.565e+03, 3.500e+00]])

In [87]:
ratings_table = ratings.pivot_table(index = 'userId', columns = 'movieId', values = 'rating')

averages = ratings_table.mean(axis = 1)

normalized_ratings_table = ratings_table.apply(lambda x : x - averages[x.name], axis = 1)
normalized_ratings_table = normalized_ratings_table.fillna(0)

In [88]:
print(normalized_ratings_table)

movieId    1       2       3       4       5         6       7       8       \
userId                                                                        
1        0.000000     0.0    0.00     0.0     0.0  0.000000     0.0     0.0   
2        0.000000     0.0    0.00     0.0     0.0  0.000000     0.0     0.0   
3        0.000000     0.0    0.00     0.0     0.0  0.000000     0.0     0.0   
4        0.000000     0.0    0.00     0.0     0.0  0.000000     0.0     0.0   
5        0.000000     0.0    0.09     0.0     0.0  0.000000     0.0     0.0   
...           ...     ...     ...     ...     ...       ...     ...     ...   
667      0.000000     0.0    0.00     0.0     0.0  0.352941     0.0     0.0   
668      0.000000     0.0    0.00     0.0     0.0  0.000000     0.0     0.0   
669      0.000000     0.0    0.00     0.0     0.0  0.000000     0.0     0.0   
670      0.193548     0.0    0.00     0.0     0.0  0.000000     0.0     0.0   
671      1.082609     0.0    0.00     0.0     0.0  0

In [89]:
R = normalized_ratings_table.values

In [90]:
keys, values = normalized_ratings_table.columns, (np.arange(9066)+1)

new_id = dict(zip(keys, values))
new_id

normalized_ratings_table.columns = np.arange(9066)+1

In [91]:
print(normalized_ratings_table)

            1     2     3     4     5         6     7     8     9     \
userId                                                                 
1       0.000000   0.0  0.00   0.0   0.0  0.000000   0.0   0.0   0.0   
2       0.000000   0.0  0.00   0.0   0.0  0.000000   0.0   0.0   0.0   
3       0.000000   0.0  0.00   0.0   0.0  0.000000   0.0   0.0   0.0   
4       0.000000   0.0  0.00   0.0   0.0  0.000000   0.0   0.0   0.0   
5       0.000000   0.0  0.09   0.0   0.0  0.000000   0.0   0.0   0.0   
...          ...   ...   ...   ...   ...       ...   ...   ...   ...   
667     0.000000   0.0  0.00   0.0   0.0  0.352941   0.0   0.0   0.0   
668     0.000000   0.0  0.00   0.0   0.0  0.000000   0.0   0.0   0.0   
669     0.000000   0.0  0.00   0.0   0.0  0.000000   0.0   0.0   0.0   
670     0.193548   0.0  0.00   0.0   0.0  0.000000   0.0   0.0   0.0   
671     1.082609   0.0  0.00   0.0   0.0  0.000000   0.0   0.0   0.0   

            10    ...  9057  9058  9059  9060  9061  9062  9063

In [92]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(ratings_set, test_size=0.2, random_state=42)

In [76]:
def SGD():
    
    alpha = 0.01
    epochs = 10
    l_factors = 50
    
    p = np.random.normal(0, .1, (R.shape[0], l_factors))
    q = np.random.normal(0, .1, (R.shape[1], l_factors))
    
    for i in range(epochs):
        for j in range(train.shape[0]):
                user, item, rating = train[j]
                user = user - 1
                item = new_id[item] - 1
                #print(user ,item , rating)
                #print(type(user.astype(np.int32)))
                error = rating - np.dot(p[user.astype(np.int32)], q[item.astype(np.int32)])
                p[user.astype(np.int32)] += alpha * error * q[item.astype(np.int32)]
                q[item.astype(np.int32)] += alpha * error * p[user.astype(np.int32)]
    return p,q
                

In [77]:
P, Q = SGD()

In [79]:
def pred(u,i):
    return np.dot(P[u-1],Q[i-1])

In [111]:
pred(2,1)

1.8108495739044483

In [110]:
prediction = []

for i in range(len(test)):
    print(test[i][2],pred(test[i][0].astype(np.int32), new_id[test[i][1]].astype(np.int32)))
    prediction.append(pred(test[i][0].astype(np.int32), new_id[test[i][1]].astype(np.int32)))

5.0 4.836629912057282
1.0 0.35702401129796546
3.0 3.1061884342484567
4.0 3.0530661949250097
4.0 3.569468798013897
5.0 5.015058303696979
5.0 3.3711335937434894
4.0 1.9116736052398682
5.0 4.8159417065415795
3.0 2.3975022679367175
4.0 3.371949442890839
2.0 1.9903520114456938
3.0 3.3235652470883505
4.5 5.34700491683074
3.0 1.8190495263595428
4.5 4.462495278902588
4.0 4.081478623151764
2.5 3.5682874818877055
4.5 4.206227106401229
2.5 2.450080694756005
4.0 2.6929071567361937
4.0 3.2923169913630086
5.0 2.971622623313967
3.5 2.4829107926738043
3.0 4.147176738139306
4.0 3.926639385463754
3.5 3.560892154459134
5.0 1.4387693595175626
4.0 4.252099956773601
4.5 3.2385928185203516
4.0 3.859451769660484
2.5 1.8968989900817328
4.5 4.017244234868246
4.0 3.428506775879544
5.0 5.05142905548633
2.5 2.2038819172812563
4.0 3.2704654258584505
2.0 1.8890194932760886
3.0 3.119153578548529
4.0 3.275299541138588
5.0 3.6672573815617646
5.0 3.1777075700104676
3.0 3.683248581991644
3.0 2.926508515532185
4.5 3.72366

5.0 4.422855171028141
4.0 1.7508941098959816
2.0 2.0337806503255007
4.0 3.583347823765571
4.0 3.9738903610732352
4.5 4.071843478253657
4.0 3.0605592812778566
2.5 2.254028125837949
2.0 1.931033667481915
2.0 0.9258289050628157
4.0 3.6091788238858795
3.5 4.4414616542682435
5.0 4.692816545514283
4.5 4.794242498674997
3.0 2.0765415339643942
3.0 2.0519181552888606
2.5 2.110695794759384
4.5 1.7668005609606106
3.0 3.0976092973393943
3.0 3.637194660374085
5.0 5.044722642585771
4.0 3.670440316629082
3.5 3.349191429418195
2.5 2.1559603943221806
3.5 2.4493489678246303
2.0 1.6543358808090023
4.5 2.439049986917537
5.0 4.183265816286828
3.0 2.5685402855900934
3.0 4.272041347248845
4.0 2.5684914918388837
4.0 3.162335190046509
4.0 2.4727841685907155
3.0 1.3590001125039977
4.5 4.072282492488937
3.0 3.716613685848054
2.5 1.1866631354075674
4.5 4.14138220366086
5.0 2.9353747964832833
3.0 2.5203361779762354
3.5 4.260154792937235
4.5 4.112294597298285
5.0 4.166261491317233
3.0 2.134876161030116
5.0 4.304401

4.5 3.8279540915981856
4.0 4.283622886983155
3.5 3.1937066918665753
2.5 2.897273179410204
4.0 2.7817276108453455
4.0 2.1508136574199215
3.5 2.6399211650805765
4.0 4.179282427003554
1.5 2.7516837012830404
4.0 4.124375236886316
4.0 1.7340929912128016
3.5 3.4720157274105663
2.0 2.8878307509713546
4.0 4.0374674182224055
4.0 4.112506140566114
3.5 3.444217646083373
4.0 3.591541187140974
5.0 3.4137098806747734
3.0 1.7921722095601926
4.0 2.392888980834204
4.5 4.7287464381863735
3.5 3.575995323855496
2.5 1.6979523955638298
3.0 1.3596717477962115
1.0 1.9437924025809035
4.0 4.269786386370373
4.0 0.6545704750470804
3.5 2.577166131309449
5.0 4.104940463807039
3.0 2.678859385198495
3.0 1.8678384150070144
4.0 2.1783905033867876
4.0 3.9940335043685375
4.0 4.3347749234561
5.0 4.883635521123438
4.0 3.2494819105381927
3.5 3.739709893157571
5.0 5.2139970147951615
3.0 2.145777540372984
4.0 2.72885451585486
4.0 5.167239540267422
3.5 3.1958112374027685
3.5 3.284721627999796
5.0 4.829592576200651
4.0 4.603250

2.5 1.8200620205575242
4.0 4.85685337462772
4.5 4.734602992844965
3.0 2.776520142322663
4.0 3.703093663371531
4.0 3.6485795539056416
4.0 5.951502400181595
3.0 2.963849489837997
3.0 3.0915580797429043
4.0 3.42363273664912
4.5 3.873830862240841
4.0 2.759101783264945
4.0 4.105978263585644
3.0 3.3683992156203515
2.5 3.3883712770838583
2.0 2.075584825612861
3.0 3.6731938372015693
3.0 3.7085929946458647
4.0 4.799989529554828
3.0 3.3509459533879715
1.5 1.166826427761372
4.0 4.629199064380393
4.0 2.5743344008366296
4.0 3.7477878072853703
3.5 3.481242789785533
3.0 3.811541767181429
4.0 2.4092468833560927
5.0 4.13301856439722
4.5 2.3422021946023808
4.0 4.916831797914925
3.0 2.9531465895483064
3.5 3.0862289798967883
4.0 4.103048726600243
2.0 2.7963496854389693
4.0 3.995078283469982
4.0 3.8722017315785813
3.0 3.6202527908354134
3.5 2.904469416908188
3.0 2.6793197431982305
2.0 1.4718759165754431
5.0 4.449114519096309
4.5 3.651603381714771
5.0 3.5319434799401512
5.0 3.836940578730928
4.0 3.250101951

4.5 4.275211165448714
4.0 3.9538638048873893
4.0 4.692350876135582
3.0 2.9544592968094676
4.0 4.232806853989607
1.0 1.2501006938325485
5.0 4.711865376183413
4.0 2.7929910282269304
4.0 3.558087985991326
4.0 3.7406032304380994
3.0 1.2784689201932573
4.0 2.698625020695577
3.0 2.932769851992919
5.0 4.400886336724241
3.0 2.7556628544040906
5.0 2.252145687140375
3.5 3.467343006840829
2.0 3.4776039349111216
3.0 3.685498088997026
4.0 4.07736341842385
2.0 3.3888881455294326
4.0 2.848998475101606
3.0 2.8897639762297516
5.0 5.049659269737666
0.5 0.14774768143971706
3.0 2.224577481812653
4.0 4.084928972678582
5.0 3.968192686724799
3.0 3.584106645351654
5.0 5.337010558776427
4.0 2.46126511224574
4.0 3.053847530292979
4.5 4.097728939289543
4.0 3.9031481650661743
3.0 3.1610564307384243
3.0 2.9775076301743275
1.0 1.106381824560623
5.0 4.157222254604649
4.0 3.9098169781450216
5.0 3.344716248591043
4.0 4.232524745606565
4.0 3.847631120316282
2.5 2.8682148699814918
4.0 2.636005720736932
2.0 0.49234578508

...

5.0 3.1392509228965406
3.5 2.330543537311598
5.0 5.265003419717773
4.5 3.7330883749905537
2.0 2.051445183639743
2.0 2.452619051908858
2.5 1.9364220156589098
1.0 1.964630872956789
4.5 4.237949784436142
4.0 3.7355956373375307
4.5 4.616750821341334
3.0 4.000966612041588
2.0 2.525579568673784
4.0 4.135688563586933
5.0 1.8438336710466836
5.0 4.179296490153613
3.5 3.4149139465357488
3.0 2.617171164068904
4.0 3.321167884212653
1.0 1.5013420147748997
2.0 2.380244302737821
5.0 4.553280643861095
2.0 2.5503012891260854
4.5 4.835866990413759
3.0 1.767732860585278
4.0 4.517415623096335
2.0 1.4477726243061095
1.5 0.8319678244212025
4.5 4.217809781115125
1.5 0.9942043136680767
3.5 2.821904943470063
5.0 4.083347655149989
5.0 5.087647014656531
3.0 3.10547708529582
3.0 1.5785983603081901
3.5 4.037801438402393
4.0 2.0785787619063325
3.0 3.1875805144083995
4.0 2.1259531030835097
5.0 4.19099508653602
4.0 4.233596542245152
5.0 4.5359254163509535
3.0 2.709576918263598
5.0 4.986713125739043
4.5 4.415745851808

4.5 4.437110564773551
4.0 2.839335156292087
3.5 1.5633498614401204
1.0 1.324626060795932
1.0 1.4401964277351633
4.0 4.5348265192923645
2.0 1.6770187660239064
5.0 3.106203842022182
4.0 4.169819551771503
4.0 3.8789670118556985
3.0 2.410387755008308
4.0 2.309792264304936
4.5 3.9268213719773564
5.0 1.285859781388606
4.0 3.95545135984259
3.0 2.256472218037281
4.0 3.970284943941396
3.5 2.940332726028189
3.0 2.653930184880135
4.5 4.231126470767279
2.5 1.5364247779544447
4.0 4.17703677266037
5.0 5.056057140970531
4.0 2.120656935105498
2.5 2.5757003954910895
3.0 2.0924730580500484
2.5 2.397118387230366
4.0 3.677233725607195
2.5 3.2198238478986294
2.5 1.3787740980533454
2.5 1.8527100378598325
4.0 3.742133678068288
3.5 3.3615894806552165
4.0 4.497372416219716
4.5 3.767528976898198
4.0 3.0273758544385077
3.0 3.587312255908095
5.0 4.32435989572557
4.0 3.8829412934299476
2.0 2.5778480164574833
4.0 4.0596073181828505
3.0 2.281351972801069
4.0 4.034985469618244
4.5 2.273813535531817
1.0 2.272492684195

5.0 4.47919242943748
4.0 4.267247054202136
4.0 4.288901276819904
5.0 4.1933157216048045
3.0 3.074998848207883
5.0 2.001958806732297
3.0 2.7134871713983286
2.0 0.8549527952423857
1.0 1.5148689260375428
4.0 3.4183808116496683
1.0 1.8416081451198278
3.5 3.6126676081723126
1.5 2.4997770533380956
3.5 1.968116076140787
5.0 4.444254805201247
4.0 1.8915059457057644
5.0 4.9554478600521135
3.0 3.013285493547132
2.5 2.262510100296159
3.0 3.338042866003904
3.0 2.963495118108125
4.0 1.8283750161487804
3.0 3.3370470639338095
3.0 3.0602467329937815
3.0 2.9480245940001413
4.5 3.9065505669406866
4.0 3.730865097895881
3.0 2.846868926274289
5.0 4.427716137850955
5.0 2.085372562305654
3.0 3.422965889650414
1.0 0.48870366577832447
3.5 3.6413933132592256
2.5 3.6234846258222357
3.0 3.412378399346301
4.0 2.587303654685761
4.5 2.7556940149953277
5.0 4.646030962887875
3.5 3.2377007096615262
3.0 3.1543663865045866
2.5 2.4321027203893255
3.0 3.141985163303965
4.5 3.950009948353363
2.0 1.0299044348981945
4.0 3.164

4.0 2.293355471262427
2.5 1.7855104062310847
2.0 2.0983155673013987
1.0 2.116382965252719
2.5 1.8102072445705384
5.0 3.829221416767483
4.0 4.0295659914678605
3.0 3.6271853997394374
4.0 2.8415911040807944
4.0 3.7687437539855657
4.0 3.4781992013982714
3.5 3.2240083752872213
5.0 3.202616622496364
5.0 2.8481719593537016
4.0 3.916015493047039
3.0 2.905681405455835
4.0 3.590488723368257
3.5 4.256951072586405
3.5 2.8337947472110825
4.0 2.9455037610461865
4.0 4.002320100219332
5.0 5.225993569343617
5.0 2.9762206850577146
3.0 1.4240430005181477
4.0 3.369312857256082
2.5 2.092699007513456
4.5 4.118097504345826
3.5 3.6482607211026608
5.0 4.187196566229957
3.0 3.3858378242550913
3.5 3.403493679406398
2.5 2.0974730896897262
2.5 1.6947054154292414
3.0 2.2932865746123676
4.0 4.354615754971886
3.0 3.0819505316068008
5.0 2.62800613222355
4.0 3.8103648681949314
3.0 3.1806318416818526
3.0 2.8492838750521443
4.0 4.231193475151928
3.5 2.4063720253505116
4.0 4.334787670603362
4.0 4.949541236475226
5.0 3.997

1.0 1.4163382578688173
4.0 2.657620355462127
0.5 0.4814753150216223
2.0 2.655562288999745
5.0 4.84437154858441
3.0 2.277408431250826
3.0 3.2834182475147338
5.0 5.047690553979645
3.5 3.5596018066334274
5.0 4.911058060202013
4.0 3.314947333689782
4.0 3.793544659024242
2.0 2.515300552800954
4.0 2.9926832928346556
4.0 2.57411136472101
4.0 3.1585986955163388
3.0 3.677224376954631
4.0 2.2101862359921878
3.0 3.425767112222585
5.0 4.003916969762211
3.0 2.6429009929538765
5.0 3.8099613302132393
3.0 2.625291208105366
4.0 3.910572138841591
3.5 4.458724264021954
3.0 2.396251114923326
4.0 3.794273487455172
3.5 2.3741149089674964
4.0 4.219530526681237
4.0 3.7004029677664345
4.0 3.080120828712862
3.0 2.4878009184462515
5.0 4.577259435169526
4.0 4.193978580515011
5.0 3.894163672989357
2.0 2.1909138740567315
3.0 3.0720912736748174
1.0 2.5419247924598576
2.0 1.7829818844691532
0.5 1.8744013020398875
3.0 2.376800979170363
4.0 3.89808222437712
4.0 3.360540790586867
4.0 4.7164358779985065
3.0 3.50732394139

In [108]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(test[:,2],prediction)
print(mse)
rmse = np.sqrt(mse)
print(rmse)

0.8467282508146903
0.9201783798887531
